In [ ]:
############ YAHOO FINANCE API #########

In [ ]:

def fetch_stock_data(symbol, interval='2m', range_span='1d'):
    """
    Fetches stock data for a given symbol from Yahoo Finance.

    :param symbol: The stock symbol (e.g., 'NAUKRI.NS')
    :param interval: The interval for the stock data (default is '2m')
    :param range_span: The date range for the stock data (default is '1d')
    :return: A dictionary with stock data.
    """
    YAHOO_FINANCE_URL = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}"

    params = {
        "region": "US",
        "lang": "en-US",
        "includePrePost": "false",
        "interval": interval,
        "useYfid": "true",
        "range": range_span,
        "corsDomain": "finance.yahoo.com",
        ".tsrc": "finance"
    }

    response = requests.get(YAHOO_FINANCE_URL, params=params)
    status_code = response.status_code

    if status_code != 200:
        raise RuntimeError(f"An error occurred. HTTP Code: {status_code}.")
    else:
        return response.json()

# Example usage
stock_symbol = 'NAUKRI.NS'  # Example stock symbol

try:
    stock_data = fetch_stock_data(stock_symbol)
    # Process and print relevant information from stock_data
    print(stock_data)
except RuntimeError as e:
    print(f"Error fetching data for {stock_symbol}: {e}")



In [ ]:
############ ALPHA VANTAGE API #############

In [ ]:
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime
from config import ALPHA_VANTAGE_API_KEY  

def get_stock_data(symbol):
    app = TimeSeries('ALPHA_VANTAGE_API_KEY')
    data, meta_data = app.get_daily_adjusted(symbol, outputsize='compact')

    today = datetime.now().strftime('%Y-%m-%d')  # Get today's date in YYYY-MM-DD format

    if today in data:
        return {
            'symbol': symbol,
            'data': data[today]
        }
    else:
        print(f"No data available for {symbol} on {today}")
        return None

# Retrieve and store stock data for AAPL, META, and TSLA in an array
stock_prices_today = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices_today.append(stock_data)

# Displaying the array of stock prices
print(stock_prices_today)


In [ ]:
############ ALPHA VANTAGE API with requests #############

In [ ]:
import requests
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY  # Importing API key from config
import pprint  # Import pprint for pretty-printing

def get_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,  # Using imported API key
        'outputsize': 'compact'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate yesterday's date in YYYY-MM-DD format
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data and yesterday in data['Time Series (Daily)']:
        # Pretty-print only yesterday's data
        pprint.pprint({yesterday: data['Time Series (Daily)'][yesterday]})

        return {
            'symbol': symbol,
            'data': data['Time Series (Daily)'][yesterday]
        }
    else:
        print(f"No data available for {symbol} on {yesterday}")
        return None

# Retrieve and store previous day's stock data for AAPL, META, and TSLA in an array
stock_prices_yesterday = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices_yesterday.append(stock_data)

# Displaying the array of stock prices for the previous day
print(stock_prices_yesterday)


No data available for AAPL on 2023-12-02
No data available for META on 2023-12-02
No data available for TSLA on 2023-12-02
[]


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY  # Importing API key from config

def get_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,  # Using imported API key
        'outputsize': 'compact'
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate yesterday's date in YYYY-MM-DD format
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data and yesterday in data['Time Series (Daily)']:
        daily_data = data['Time Series (Daily)'][yesterday]
        # Flatten the data and add the symbol
        flattened_data = {f'{key.split(". ")[1]}': value for key, value in daily_data.items()}
        flattened_data['symbol'] = symbol
        flattened_data['date'] = yesterday  # Adding date for reference
        return flattened_data
    else:
        print(f"No data available for {symbol} on {yesterday}")
        return None

# Retrieve and store previous day's stock data in a DataFrame
stock_data_list = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_data_list.append(stock_data)

# Creating a DataFrame
df = pd.DataFrame(stock_data_list)

# Setting the index of the DataFrame to the symbol
df.set_index('symbol', inplace=True)

# Displaying the DataFrame
print(df)



In [ ]:
import requests
from datetime import datetime, timedelta
from config import ALPHA_VANTAGE_API_KEY
import pprint

def get_last_two_weeks_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'full'  # Change to 'full' to get more data
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    # Calculate the date two weeks ago
    two_weeks_ago = (datetime.now() - timedelta(weeks=2)).strftime('%Y-%m-%d')

    if 'Time Series (Daily)' in data:
        time_series = data['Time Series (Daily)']
        last_two_weeks_data = {date: details for date, details in time_series.items() if date >= two_weeks_ago}
        pprint.pprint(last_two_weeks_data)  # Pretty-print the two weeks' data

        return {
            'symbol': symbol,
            'data': last_two_weeks_data
        }
    else:
        print(f"No data available for {symbol}")
        return None

# Retrieve and store the last two weeks' stock data for AAPL, META, and TSLA
stock_prices_last_two_weeks = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_last_two_weeks_stock_data(symbol)
    if stock_data is not None:
        stock_prices_last_two_weeks.append(stock_data)

# Display the array of stock prices for the last two weeks
pprint.pprint(stock_prices_last_two_weeks)


In [ ]:
#### Market closed on weekends ####

In [ ]:


def get_stock_data(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'compact'
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise AlphaVantageApiException(response.status_code)

    data = response.json()
    time_series = data.get('Time Series (Daily)', {})

    # Function to get the last weekday
    def get_last_weekday(date):
        while date.weekday() > 4:  # Mon-Fri are 0-4
            date -= timedelta(days=1)
        return date

    last_weekday = get_last_weekday(datetime.now() - timedelta(days=1))

    # Check for the last available data
    while last_weekday.strftime('%Y-%m-%d') not in time_series:
        last_weekday = get_last_weekday(last_weekday - timedelta(days=1))

    last_date_str = last_weekday.strftime('%Y-%m-%d')
    if last_date_str in time_series:
        daily_data = time_series[last_date_str]
        flattened_data = {
            'date': last_date_str,
            'symbol': symbol,
            'open': daily_data.get('1. open'),
            'high': daily_data.get('2. high'),
            'low': daily_data.get('3. low'),
            'close': daily_data.get('4. close'),
            'adjusted_close': daily_data.get('5. adjusted close'),
            'volume': daily_data.get('6. volume'),
            'dividend_amount': daily_data.get('7. dividend amount'),
            'split_coefficient': daily_data.get('8. split coefficient')
        }
        return flattened_data
    else:
        return None



# Retrieve and store the last two weeks' stock data for AAPL, META, and TSLA
stock_prices = []

stock_symbols = ['AAPL', 'META', 'TSLA']
for symbol in stock_symbols:
    stock_data = get_stock_data(symbol)
    if stock_data is not None:
        stock_prices.append(stock_data)

# Display the array of stock prices for the last two weeks
pprint.pprint(stock_prices)

[{'adjusted_close': '191.24',
  'close': '191.24',
  'date': '2023-12-01',
  'dividend_amount': '0.0000',
  'high': '191.56',
  'low': '189.23',
  'open': '190.33',
  'split_coefficient': '1.0',
  'symbol': 'AAPL',
  'volume': '45704823'},
 {'adjusted_close': '324.82',
  'close': '324.82',
  'date': '2023-12-01',
  'dividend_amount': '0.0000',
  'high': '326.86',
  'low': '320.76',
  'open': '325.48',
  'split_coefficient': '1.0',
  'symbol': 'META',
  'volume': '15276375'},
 {'adjusted_close': '238.83',
  'close': '238.83',
  'date': '2023-12-01',
  'dividend_amount': '0.0000',
  'high': '240.19',
  'low': '231.9',
  'open': '233.14',
  'split_coefficient': '1.0',
  'symbol': 'TSLA',
  'volume': '121331709'}]


In [4]:
from flask import Flask, render_template, request
import requests
from config import ALPHA_VANTAGE_API_KEY
import pprint

stock_symbols = ['AAPL', 'META', 'TSLA']

def get_stock_data_for_two_weeks(symbol):
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': symbol,
        'apikey': ALPHA_VANTAGE_API_KEY,
        'outputsize': 'full'  # Changed to 'full' to get more data
    }
    response = requests.get(base_url, params=params)
    
    if response.status_code != 200:
        raise AlphaVantageApiException(response.status_code)

    data = response.json()
    time_series = data.get('Time Series (Daily)', {})

    # Sort the dates and get the last 10 trading days
    sorted_dates = sorted(time_series.keys(), reverse=True)[:10]
    last_10_days_data = [time_series[date] for date in sorted_dates]
    
    return last_10_days_data

def calculate_two_week_average(stock_data):
    total = sum(float(day_data['4. close']) for day_data in stock_data)
    average = total / len(stock_data)
    return average

# Retrieve and store the last two weeks' stock data for AAPL, META, and TSLA
stock_prices = []
for symbol in stock_symbols:
    stock_data = get_stock_data_for_two_weeks(symbol)
    if stock_data:
        current_price = float(stock_data[0]['4. close'])
        two_week_average = calculate_two_week_average(stock_data)
        stock_prices.append({
            'symbol': symbol,
            'current_price': current_price,
            'two_week_average': two_week_average
        })

# Display the array of stock prices for the last two weeks
pprint.pprint(stock_prices)


NameError: name 'requests' is not defined